<a href="https://colab.research.google.com/github/stratis-forge/radiomics-workflows/blob/main/compare_CERR_radiomics_scalar_feat_with_PyRadiomics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download latest Octave compile


In [1]:
%%capture
! apt-get update
! apt-get install libgraphicsmagick++1-dev libsuitesparse-dev libqrupdate1 \
libfftw3-3 gnuplot zsh openjdk-8-jdk
! cd /usr/lib/x86_64-linux-gnu/ && ln -s libhdf5_serial.so.103 libhdf5_serial.so.100 \
&& ln -s libreadline.so.8 libreadline.so.7
!apt-get install gfortran-7

In [2]:
#Download latest compiled octave package 
def get_octave(root_path):
  os.chdir(root_path)
  with urllib.request.urlopen("https://api.github.com/repos/cerr/octave-colab/releases/latest") as url:
      data = json.loads(url.read().decode())
  fname = data['assets'][0]['name']
  requrl = data['assets'][0]['browser_download_url']
  urllib.request.urlretrieve(requrl, fname)
  # Unzip, identify octave folder name
  !tar xf {fname}
  top_folder = !tar tf {fname} | head -1
  octave_folder = top_folder[0][:-1]
  octave_path = os.path.join(root_path,octave_folder)
  return octave_path

In [3]:
# Set path to Octave exectuable
import os, urllib.request, json
octave_path = get_octave('/content')
os.environ['OCTAVE_EXECUTABLE'] = octave_path + '/bin/octave-cli'  #Replace with OCTAVE_EXECUTABLE path
os.environ['PATH'] = octave_path + '/bin:' + os.environ['PATH']    #Replace with Octave path

In [4]:
%%capture
! pip3 install octave_kernel
! pip3 install oct2py

In [5]:
!pip install pyradiomics

## Download CERR

In [6]:
%%capture
!git clone --single-branch --branch octave_dev https://www.github.com/cerr/CERR.git  
!cd /content/CERR && git checkout 1c0b1af23b3cfbd9183f46022559b5abdbc3d93d
!cd /content

In [7]:
%load_ext oct2py.ipython
from oct2py import octave

In [8]:
%%capture
%%octave
pkg load statistics
pkg load image
pkg load io

In [9]:
%%capture
%%octave
#Add CERR to path
curr_dir = pwd()
cd('/content/CERR')
addToPath2(pwd)
cd(curr_dir)

## IBSI-1 feature calculations using CERR and PyRadiomics

### Compute first-order and texture features using CERR

In [10]:
%%octave

#Compute features using CERR

#Load sample data
fpath = fullfile(fileparts(fileparts(getCERRPath)),...
    'Unit_Testing/data_for_cerr_tests/IBSI1_CT_phantom/IBSILungCancerCTImage.mat.bz2');
planC = loadPlanC(fpath,tempdir);
planC = updatePlanFields(planC);
planC = quality_assure_planC(fpath,planC);
indexS = planC{end};
strName = 'GTV-1';

pyParamFilePath = fullfile(fileparts(fileparts(getCERRPath)),...
            'Unit_Testing/settings_for_comparisons/pyOrigNoInterp.yaml');
cerrParamFilePath = fullfile(fileparts(fileparts(getCERRPath)),...
            'Unit_Testing/settings_for_comparisons/cerrOrigNoInterp.json');
paramS = getRadiomicsParamTemplate(cerrParamFilePath);

strC = {planC{indexS.structures}.structureName};
structNum = getMatchingIndex(paramS.structuresC{1},strC,'exact');
scanNum = getStructureAssociatedScan(structNum,planC);
cerrFeatS = calcGlobalRadiomicsFeatures...
    (scanNum, structNum, paramS, planC);

filtName = fieldnames(cerrFeatS);
filtName = filtName{1};

CERR>>  Decompressing IBSILungCancerCTImage.mat.bz2...

ans = 0

CERR>>  Loading IBSILungCancerCTImage.mat.bz2...

CERR>>  Loaded IBSILungCancerCTImage.mat.bz2...

Elapsed time is 1.28746e-05 seconds.

Elapsed time is 0.618907 seconds.

Elapsed time is 0.988859 seconds.

### Compute features using PyRadiomics

#### Export scan & mask to NRRD for input to Pyradiomics

In [11]:
%%octave
function [scanFilename, maskFilename] = exportToNRRD(strName,scanNum,planC,outDir)

  # Extract scan & mask
  indexS = planC{end};
  strC = {planC{indexS.structures}.structureName};
  if isnumeric(strName)
    strNum = strName;
  else
    strNum = getMatchingIndex(strName,strC,'exact');    
  end
  mask3M = getStrMask(strNum,planC);

  if isempty(scanNum)
    scanNum = getStructureAssociatedScan(strNum,planC);
  end
  scan3M = double(getScanArray(scanNum,planC));
  CToffset = planC{indexS.scan}(scanNum).scanInfo(1).CTOffset;
  scan3M = scan3M - CToffset;

  # Get voxel size
  scanS = planC{indexS.scan}(scanNum);
  [xV,yV,zV] = getScanXYZVals(scanS);
  dx = median(abs(diff(xV)));
  dy = median(abs(diff(yV)));
  dz = median(diff(zV));
  voxelSizeV = [dx, dy, dz]*10; %convert to mm

  # Export to NRRD
  originV = [0,0,0];
  encoding = 'raw';
  mask3M = uint16(mask3M);
  mask3M = permute(mask3M, [2 1 3]);
  scan3M = permute(scan3M, [2 1 3]);
  scanFilename = fullfile(outDir,'scan.nrrd');
  scanRes = nrrdWriter(scanFilename, scan3M, voxelSizeV, originV, encoding);
  maskFilename = fullfile(outDir,'mask.nrrd');
  maskRes = nrrdWriter(maskFilename, mask3M, voxelSizeV, originV, encoding);

end

In [12]:
%%octave
outDir = '/content/tmpdir';
mkdir(outDir)
[scanFilename, maskFilename] = exportToNRRD(strName,[],planC,outDir);

#### Extract features 

In [13]:
%octave_pull scanFilename maskFilename pyParamFilePath outDir

In [14]:
import sys
sys.path.insert(0, '/content/CERR/Unit_Testing/tests_for_cerr/')
from pyFeatureExtraction import extract
pyFeatDict = extract(scanFilename, maskFilename,pyParamFilePath,outDir);

INFO:radiomics.featureextractor:Loading parameter file /content/CERR/Unit_Testing/settings_for_comparisons/pyOrigNoInterp.yaml
INFO:radiomics.featureextractor:Loading image and mask
INFO:radiomics.featureextractor:Calculating features with label: 1
INFO:radiomics.featureextractor:Loading image and mask


Reading image and mask...
Initializing feature extractor...
Done.


INFO:radiomics.featureextractor:Adding image type "Original" with custom settings: {}
INFO:radiomics.featureextractor:Calculating features for original image
INFO:radiomics.featureextractor:Computing firstorder
INFO:radiomics.featureextractor:Computing glcm
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
INFO:radiomics.featureextractor:Computing glrlm
INFO:radiomics.featureextractor:Computing glszm
INFO:radiomics.featureextractor:Computing gldm


## Compare results

### Map PyRadiomics output to corresponding CERR features

In [15]:
%octave_push pyFeatDict

In [16]:
%%octave
featS = struct(pyFeatDict);
fieldsC = fieldnames(featS);
for n=1:length(fieldsC)
   if isa(featS.(fieldsC{n}),'py.numpy.ndarray')
       featS.(fieldsC{n}) = double(featS.(fieldsC{n}));
   end
end


#Map to cerr fieldnames
pyFeatS = struct();

#First-order
pyFirstOrdFeatS = getPyradFeatDict(featS,{['original','_firstorder']});
pyFirstOrdFeatS = mapPyradFieldnames(pyFirstOrdFeatS,'original','firstorder');
pyFeatS.(filtName).firstOrderS = pyFirstOrdFeatS;

#GLCM
pyGlcmFeatS = getPyradFeatDict(featS,{['original','_glcm']});
pyGlcmFeatS = mapPyradFieldnames(pyGlcmFeatS,'original','glcm');
pyFeatS.(filtName).glcmFeatS = pyGlcmFeatS;

#GLRLM
pyGlrlmFeatS = getPyradFeatDict(featS,{['original','_glrlm']});
pyGlrlmFeatS = mapPyradFieldnames(pyGlrlmFeatS,'original','glrlm');
pyFeatS.(filtName).rlmFeatS = pyGlrlmFeatS;

#NGLDM
pyGldmFeatS = getPyradFeatDict(featS,{['original','_gldm']});
pyGldmFeatS = mapPyradFieldnames(pyGldmFeatS,'original','ngldm');
pyFeatS.(filtName).ngldmFeatS = pyGldmFeatS;

#GLSZM
pyGlszmFeatS = getPyradFeatDict(featS,{['original','_glszm']});
pyGlszmFeatS = mapPyradFieldnames(pyGlszmFeatS,'original','glszm');
pyFeatS.(filtName).szmFeatS = pyGlszmFeatS;


#### E.g. Compare first-order statistics

In [17]:
%%octave

# First-order intensity features
firstOrdFeat1C = fieldnames(cerrFeatS.Original.firstOrderS);
firstOrdFeat2C = fieldnames(pyFeatS.Original.firstOrderS);

allFeatC = unique([firstOrdFeat1C;firstOrdFeat2C]);
keepIdxV = ismember(allFeatC,firstOrdFeat1C) & ismember(allFeatC,firstOrdFeat2C);
selFeatC = allFeatC(keepIdxV);

for n = 1:length(selFeatC)
   cerrVal = cerrFeatS.Original.firstOrderS.(selFeatC{n});
   pyVal = pyFeatS.Original.firstOrderS.(selFeatC{n});
   pctDiff =(cerrVal-pyVal)*100/pyVal;
   pctDiffS.FirstOrder.(selFeatC{n}) = pctDiff;
end


pctDiffS.FirstOrder

ans =

  scalar structure containing the fields:

    min = 0

    max = 0

    mean = 0

...

    rms = 0

    robustMeanAbsDev = -1.8518e-10

    skewness = -2.1577e-12

    totalEnergy = -3.9045e-07

    var = 4.9058e-13